# Lab 10.2 Accelerating Supervised Fine Tuning with Deepspeed
## (Part III Run (distributed) SFT with Deepspeed)

## 0. Dependencies and compatibility

In [1]:
!pip install -r requirements.txt

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 337.6 kB/s eta 0:00:00a 0:00:01
  Using cached fire-0.6.0-py2.py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 331.4 kB/s eta 0:00:0000:0100:10
  Using cached deepspeed-0.14.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 326.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 307.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 334.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 327.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 341.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 303.1 kB/s eta 0:00:00a 0:00:01


## 1. Create the training script
In this section, we will finetune a chat model using parallel technologies. We choose Llama-chat-7B as our base model and our dataset is code instrcutions, which guides LLM to write programming code from given natural language. 
Recall what we have learned in the lab8, now we put all steps into the launch script `launch_sft_llama2.py`

In [3]:
%%file /tmp/ds_zero2_no_offload.json

{
    "bf16": {
        "enabled": "auto"
      },

    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": true,
        "allgather_bucket_size": 1e8,
        "overlap_comm": true,
        "reduce_scatter": true,
        "reduce_bucket_size": 1e8,
        "contiguous_gradients": true
    },
    
    "checkpoint": {
        "use_node_local_storage": true
      },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}


Writing /tmp/ds_zero2_no_offload.json


In [4]:
%%file /tmp/train.py

import torch
import os

################################################################################
# Model name and directories
################################################################################

# The base model
model_name = "/share/model/llama-2-7b-chat-hf"
# The instruction dataset to use
dataset_name = "/root/tmp/py18k"
# Fine-tuned model name
new_model = "/scratch2/llama-2-7b-py18k"
# Output directory where the model predictions and checkpoints will be stored
output_dir = "/scratch2/results"

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,    # use 4-bit precision for base model loading
    bnb_4bit_quant_type= "nf4",  # Quantization type (fp4 or nf4)
    bnb_4bit_compute_dtype= torch.bfloat16,   # Compute dtype for 4-bit base models  "float16" or torch.bfloat16
    bnb_4bit_use_double_quant= False,  # Activate nested quantization for 4-bit base models (double quantization)
)

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

local_rank = int(os.environ.get("LOCAL_RANK", -1))

device_map = {'':local_rank}

print("device map: ", device_map)

# Load base model with bnb config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1
bias="none"
task_type="CAUSAL_LM"

################################################################################
# Training parameters (passed to TrainingArguments)
################################################################################

# Number of training epochs
num_train_epochs = 1
# Number of training steps (overrides num_train_epochs)
# max_steps = 100
# Enable fp16/bf16 training (set bf16 to True if supported by your GPU)
fp16 = False
bf16 = True
# Batch size per GPU for training
per_device_train_batch_size = 4
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine"
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 0

################################################################################
# Monitoring parameters
################################################################################

# Logging dir (for tensorboard)
logging_dir = f"{output_dir}/logs"
# Log every X updates steps
logging_steps = 25
# Monitoring and Visualizing tools
report_to = "tensorboard"

################################################################################
# SFTTrainer parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 512
# Pack multiple short examples in the same input sequence to increase efficiency
packing = False


# Load dataset
from datasets import load_from_disk
dataset = load_from_disk(dataset_name)
print(dataset[0])


from peft import PeftModel
from trl import SFTTrainer

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    deepspeed="/tmp/ds_zero2_no_offload.json",  ## point to the deepspeed file generated in the previous step
    #max_steps=max_steps,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    logging_steps=logging_steps,
    logging_dir=logging_dir,
    report_to=report_to, 
)

from peft import LoraConfig
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias=bias,
    task_type=task_type,
)


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# # Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)


# release GPU memory here

torch.cuda.reset_peak_memory_stats()
torch.cuda.empty_cache()

Overwriting /tmp/train.py


Similarly we can start the tensorboard for monitoring before training.

In this project we use deepspeed to speed up training, you can find details of deepspeed configuration in the `ds_zero2_no_offload.json`. 
You can adjust the number of GPUs for training according to the number of available GPUs on your POD by changing `nproc_per_node` and adjust the total number of POD invovled in the training by changing `nnodes`.

## 2. Running the training on a single machine

In [4]:
# single pod multiple GPUs
# run the following command in the terminal
# NCCL_P2P_DISABLE=1 NCCL_IB_DISABLE=1 torchrun --nnodes 1 --nproc_per_node 2 /tmp/train.py

In [ ]:
# You can use the tensorboard to monitor the training process, as before

# tensorboard --logdir /scratch2/results/logs

## 3. Running the script in a distributed way

To start training on multiple pods, select one of your pods as the master node and check its IP of the infiniband NIC using `ifconfig`.  

In [3]:
# find the IP address

!ifconfig

cali27c024edcba: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1450
        inet6 fe80::ecee:eeff:feee:eeee  prefixlen 64  scopeid 0x20<link>
        ether ee:ee:ee:ee:ee:ee  txqueuelen 1000  (Ethernet)
        RX packets 57  bytes 4086 (4.0 KB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 68  bytes 7749 (7.7 KB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

cali783eeef06f8: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1450
        inet6 fe80::ecee:eeff:feee:eeee  prefixlen 64  scopeid 0x20<link>
        ether ee:ee:ee:ee:ee:ee  txqueuelen 1000  (Ethernet)
        RX packets 10982943  bytes 814076038 (814.0 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 16100644  bytes 35129496214 (35.1 GB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

cali9645486a5d5: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1450
        inet6 fe80::ecee:eeff:feee:eeee  prefixlen 64  scopeid 0x20<link>
       

In [6]:
# multiple pod multiple GPUs
# run the following command in the terminal 
# (you need to set the rank to 0 or 1 for each pod, (master is 0), also set ehe msater IP addresss)
# you should find the master_ip address by running ifconfig and look for the IP address starting with 10.3.x.x (200Gbps IB)

# NCCL_P2P_DISABLE=1 NCCL_IB_DISABLE=1 torchrun --nnodes 2 --nproc_per_node 2 --node_rank 0 --master_addr 10.3.0.0 /tmp/train.py

In [7]:
# On the master node only, run the following in the terminal to start the tensorboard

# tensorboard --logdir /scratch2/results/logs

## 4 Merge Lora_model with Base model and save the merged model

You should first export Lora model with base model and convert them into hf checkpoint. 
This makes up the final trained merged model.

In [4]:
################################################################################
# Shared parameters between inference and SFT training
################################################################################
import torch

# The base model
model_name = "/share/model/llama-2-7b-chat-hf"


################################################################################
# bitsandbytes parameters
################################################################################
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,    # use 4-bit precision for base model loading
    bnb_4bit_quant_type= "nf4",  # Quantization type (fp4 or nf4)
    bnb_4bit_compute_dtype= torch.bfloat16,   # Compute dtype for 4-bit base models  "float16" or torch.bfloat16
    bnb_4bit_use_double_quant= False,  # Activate nested quantization for 4-bit base models (double quantization)
)

In [9]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [10]:
from peft import PeftModel

new_model = "/scratch2/llama-2-7b-py18k"
device_map = "auto"

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
merged_model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Save merged model to disk (optional)
merged_model.save_pretrained(f'{new_model}_merged')


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## 5 examine the results in tensorboard


In [11]:
# set up port forwarding in vscode
# open the tensorboard page at http://localhost:6006

## 6 Test the SFTed model

In [25]:
# Run text generation pipeline with our next model
prompt = "Write a Python function to return the mode (the value or values that appear most frequently within that list) in a given list. If there are multiple modes, return -1. You should generate a function:\n def solve(list):"
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer, temperature=0.3, do_sample=True, max_length=1024)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Write a Python function to return the mode (the value or values that appear most frequently within that list) in a given list. If there are multiple modes, return -1. You should generate a function:
 def solve(list): [/INST] ```def solve(list):
    count = {}
    for item in list:
        count[item] = count.get(item, 0) + 1
    max_count = max(count.values())
    max_count_items = [item for item, count in count.items() if count == max_count]
    if len(max_count_items) > 1:
        return -1
    else:
        return max_count_items[0]

list = [1, 2, 3, 3, 4, 5, 5, 5, 6, 6, 6, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9

In [21]:
# Using regrex to capture the generated Python to a string
import re

def extract_first_code_snippet(text):
    # Use a regular expression to find the first code snippet enclosed in triple backticks
    match = re.search(r"```(.*?)```", text, re.S)
    if match:
        # Return the first matched group, which is the content within the backticks
        return match.group(1)
    else:
        # Return None if no match is found
        return None

In [22]:
# Define a testcase using the standard python unittest library

import unittest

# place holder for the AI generated code
def solve(list):
    return 0  

class TestGeneratedCode(unittest.TestCase):

    def test_no_single_mode(self):
        self.assertEqual(solve([3, 2, 1]), -1)

    def test_single_mode(self):
        self.assertEqual(solve([4, 9, 2, 33, 2]), 2)

    def test_no_single_mode_3(self):
        self.assertEqual(solve([7, 9, 11, 323, 996]), -1)

def run_all_tests():
    unittest.main(argv=[''], verbosity=2, exit=False) 

In [23]:
code = extract_first_code_snippet(result[0]['generated_text'])
print(code)


def solve(list):
  # calculate the number of occurrences of each element in list
  occurrences = collections.Counter(list)
  # calculate the mode
  mode = occurrences.most_common()[0][0]
  # if there is only one mode
  return mode if occurrences.most_common()[0][0] == 1 else -1

# Test
print(solve([1, 2, 3, 4, 2, 3, 4]))


In [24]:
exec(code)
run_all_tests() # Expect to pass

NameError: name 'collections' is not defined

## 7 (optional) Experience Zero-3 
Now you can try DeepSpeed with a more aggressive strategy named `stage-3` or `zero-3` by replacing the DeepSpeed configuration file using `ds_zero3_offload_pt.json`. Rerun the training and observe the changes on GPU memory usage and training time.